# Download BERPublicsearch.zip, unzip & convert to parquet

In [10]:
!pip install geopandas seaborn matplotlib descartes seai-deap git+https://github.com/codema-dev/ber-public

  Cloning https://github.com/codema-dev/ber-public to /tmp/pip-req-build-dovphlgk
  Running command git clone -q https://github.com/codema-dev/ber-public /tmp/pip-req-build-dovphlgk
^C
ERROR: Operation cancelled by user


In [ ]:
from ber_public.download import download_berpublicsearch_parquet

# Register your email address with SEAI at https://ndber.seai.ie/BERResearchTool/Register/Register.aspx
email_address="EMAIL-ADDRESS"
download_berpublicsearch_parquet(email_address)

# Calculate the Heat Loss Parameter for the `BER Public search` dataset

... to estimate the number of households currently eligible for Heat Pumps

In [1]:
import dask.dataframe as dd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from ber_public import archetype
from ber_public.deap import dim
from ber_public.deap import fab
from ber_public.deap import vent

In [2]:
columns = [
    'CountyName',
    
    # Area
	'WallArea',
	'RoofArea',
	'FloorArea',
	'WindowArea',
	'DoorArea',
	'GroundFloorArea',
	'FirstFloorArea',
	'SecondFloorArea',
	'ThirdFloorArea',
 
	# Floor Heights
	'GroundFloorHeight',
	'FirstFloorHeight',
	'SecondFloorHeight',
	'ThirdFloorHeight',
 
	# Uvalues
	'UValueWall',
	'UValueRoof',
	'UValueFloor',
	'UValueWindow',
	'UvalueDoor',
 
 	# Ventilation
  	'NoOfSidesSheltered',
   	'NoOfChimneys',
    'NoOfOpenFlues',
    'NoOfFansAndVents',
	'NoOfFluelessGasFires',
 	'DraftLobby',
	'PermeabilityTest',
	'PermeabilityTestResult',
	'NoStoreys',
 	'PercentageDraughtStripped',
	'SuspendedWoodenFloor',
	'StructureType',
	'VentilationMethod',
	'HeatExchangerEff',
]

In [3]:
ireland_bers = dd.read_parquet("BERPublicsearch_parquet", columns=columns)

In [4]:
# extract Dublin BERs and convert from dask to pandas via compute()
building_stock = (
    ireland_bers[ireland_bers["CountyName"].str.contains('Dublin')].compute()
    .reset_index(drop=True)
)

# Calculate HLP as in DEAP 4.2.2

In [5]:
building_volume = (
    building_stock["GroundFloorArea"].fillna(0) * building_stock["GroundFloorHeight"].fillna(0)
    + building_stock["FirstFloorArea"].fillna(0) * building_stock["FirstFloorHeight"].fillna(0)
    + building_stock["SecondFloorArea"].fillna(0) * building_stock["SecondFloorHeight"].fillna(0)
    + building_stock["ThirdFloorArea"].fillna(0) * building_stock["ThirdFloorHeight"].fillna(0)
)

In [6]:
fabric_heat_loss = fab.calculate_fabric_heat_loss(
	roof_area=building_stock["RoofArea"],
    roof_uvalue=building_stock["UValueRoof"],
    wall_area=building_stock["WallArea"],
    wall_uvalue=building_stock["UValueWall"],
    floor_area=building_stock["FloorArea"],
    floor_uvalue=building_stock["UValueFloor"],
    window_area=building_stock["WindowArea"],
    window_uvalue=building_stock["UValueWindow"],
    door_area=building_stock["DoorArea"],
    door_uvalue=building_stock["UvalueDoor"],
    thermal_bridging_factor=0.05,
)

In [7]:
infiltration_rate = vent.calculate_infiltration_rate(
    no_sides_sheltered=building_stock["NoOfSidesSheltered"],
    building_volume=building_volume,
    no_chimneys=building_stock["NoOfChimneys"],
    no_open_flues=building_stock["NoOfOpenFlues"],
    no_fans=building_stock["NoOfFansAndVents"],
    no_room_heaters=building_stock["NoOfFluelessGasFires"],
    is_draught_lobby=building_stock["DraftLobby"],
    is_permeability_tested=building_stock["PermeabilityTest"],
    permeability_test_result=building_stock["PermeabilityTestResult"],
    no_storeys=building_stock["NoStoreys"],
    percentage_draught_stripped=building_stock["PercentageDraughtStripped"],
    is_floor_suspended=building_stock["SuspendedWoodenFloor"],
    structure_type=building_stock["StructureType"],
)

In [8]:
effective_air_rate_change = vent.calculate_effective_air_rate_change(
	ventilation_method=building_stock["VentilationMethod"],
    building_volume=building_volume,
    infiltration_rate=infiltration_rate,
    heat_exchanger_efficiency=building_stock["HeatExchangerEff"],
)

In [9]:
ventilation_heat_loss = vent.calculate_ventilation_heat_loss(
	building_volume=building_volume,
	effective_air_rate_change=effective_air_rate_change,
	ventilation_heat_loss_constant=0.33,
)

In [14]:
total_floor_area = (
	building_stock["GroundFloorArea"].fillna(0)
    + building_stock["FirstFloorArea"].fillna(0)
	+ building_stock["SecondFloorArea"].fillna(0)
    + building_stock["ThirdFloorArea"].fillna(0)
)

In [16]:
heat_loss_coefficient = fabric_heat_loss + ventilation_heat_loss

In [23]:
assert total_floor_area[total_floor_area.isna()].empty
heat_loss_parameter = heat_loss_coefficient / total_floor_area